In [ ]:
import numpy as np
import pandas as pd

import os
import seaborn as sns
import json
import ast
import glob
from tqdm import tqdm
import gc

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pydicom
import cv2
from pydicom.pixel_data_handlers.util import apply_voi_lut

import random

In [ ]:
train_csv = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
display(train_csv.head())

# draw .dcm image function

## 1.glob.glob -> get image path 
## 2.sorted (data,sorting creteria)
## 3.pydicom -> read image
## 4.plt.imshow -> draw image

In [ ]:
KAGGLE_DIR = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
IMG_PATH_TRAIN = KAGGLE_DIR + 'train/'

def read_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0 :
        data = data/np.max(data)
    data = (data*255.0).astype(np.uint8)
    return data

def visualize_sample(brats21id, slice_i,mgmt_value,types=("FLAIR", "T1w", "T1wCE", "T2w")):
    plt.figure(figsize=(16, 5))
    patient_path = os.path.join(IMG_PATH_TRAIN,str(brats21id).zfill(5),)
    print("patient_path",patient_path)
    for i, t in enumerate(types, 1):
#         print("glob data = ", glob.glob(os.path.join(patient_path, t, "*")))
        t_paths = sorted(
            glob.glob(os.path.join(patient_path, t, "*")), 
            key=lambda x: int(x[:-4].split("-")[-1]),
        )
#         print("image full path=",os.path.join(patient_path, t, "*"))
#         print("t_paths=",t_paths)
        data = read_dicom(t_paths[int(len(t_paths) * slice_i)])
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")

    plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.savefig('plot.png')
    plt.show()

# sort by filename  

## glob.glob -> not sorted 

In [ ]:
brats21id = train_csv.iloc[0]["BraTS21ID"] 
mgmt_value = train_csv.iloc[0]["MGMT_value"] # and tumor class
patient_path = os.path.join(IMG_PATH_TRAIN,str(brats21id).zfill(5),)
x = glob.glob(os.path.join(patient_path, "FLAIR", "*"))
x[:5]

## sorted (path,sort by last filename) 

In [ ]:
t_paths = sorted(x,key=lambda x: int(x[:-4].split("-")[-1]))
t_paths[:5]

## split filepath -> distinct filename  

In [ ]:
x[0][:-4].split("-")[-1]

## draw sample

In [ ]:
_brats21id = train_csv.iloc[0]["BraTS21ID"] 
_mgmt_value = train_csv.iloc[0]["MGMT_value"] # and tumor class
print("BraTS21ID=",_brats21id)
print("MGMT_value=",_mgmt_value)
visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5) # visualize samples